In [1]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras import optimizers
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import tensorflow as tf

Using TensorFlow backend.


Xception for 
apple_pie/       beef_tartare/  bread_pudding/      cannoli/
baby_back_ribs/  beet_salad/    breakfast_burrito/  caprese_salad/
baklava/         beignets/      bruschetta/
beef_carpaccio/  bibimbap/      caesar_salad/

In [3]:
ls newprueba

ramen/  red_velvet_cake/  samosa/  sashimi/  seaweed_salad/


In [4]:
train_data_dir='newprueba'


In [5]:
TEST_SPLIT = 0.25
epochs = 80
#batch_size = 16
batch_size=32
MICRO_DATA = True # very small subset (just 3 groups)
SAMPLE_TRAINING = False # make train set smaller for faster iteration
IMG_SIZE = (112, 112)
LEARNING_RATE = 0.001

In [4]:

train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 11251 images belonging to 15 classes.
Found 2811 images belonging to 15 classes.


In [26]:
validation_generator

In [6]:
train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

Found 3984 images belonging to 5 classes.
Found 995 images belonging to 5 classes.


In [7]:
from keras import applications
model = applications.Xception(weights='imagenet', 
                                include_top=False, input_shape = (112,112,3))

In [8]:
len(model.layers)

132

In [12]:
for layer in model.layers[:40]:
    layer.trainable = False
regparameter=0.0001

x = model.output
x = Flatten()(x)
x = Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.l1_l2(l1=regparameter,l2=regparameter))(x)
x = Dropout(0.4)(x)
x = Dense(4096, activation="relu")(x)
predictions = Dense(5, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])
model_final.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 55, 55, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 55, 55, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 55, 55, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

/home/davidrendon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]


In [13]:
chkpt = ModelCheckpoint(filepath='best_model_todatexceptiionfreeze40.hdf5', save_best_only=True, save_weights_only=True)

In [14]:
tf.keras.callbacks.ModelCheckpoint

tensorflow.python.keras.callbacks.ModelCheckpoint

In [15]:
pwd

'/home/davidrendon/Documents/proyectou3deep'

In [19]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 20,callbacks=[chkpt])

Epoch 1/20
124/124 [==============================] - 144s 1s/step - loss: 0.2919 - accuracy: 0.9810 - val_loss: 0.9382 - val_accuracy: 0.8851
Epoch 2/20
124/124 [==============================] - 143s 1s/step - loss: 0.2163 - accuracy: 0.9904 - val_loss: 0.7828 - val_accuracy: 0.8712
Epoch 3/20
124/124 [==============================] - 143s 1s/step - loss: 0.1817 - accuracy: 0.9947 - val_loss: 0.5507 - val_accuracy: 0.9065
Epoch 4/20
124/124 [==============================] - 137s 1s/step - loss: 0.1445 - accuracy: 0.9980 - val_loss: 1.2476 - val_accuracy: 0.8930
Epoch 5/20
124/124 [==============================] - 136s 1s/step - loss: 0.1356 - accuracy: 0.9960 - val_loss: 0.8805 - val_accuracy: 0.8744
Epoch 6/20
124/124 [==============================] - 137s 1s/step - loss: 0.1945 - accuracy: 0.9906 - val_loss: 1.4109 - val_accuracy: 0.9086
Epoch 7/20
124/124 [==============================] - 137s 1s/step - loss: 0.1492 - accuracy: 0.9960 - val_loss: 1.0158 - val_accuracy: 0.8723

In [22]:
model_final.save('xception5classacc59')

In [21]:
ls

amoestadb/                               newprueba/
best_model_todatefreeze50.hdf5           prueba/
best_model_todate.hdf5                   Untitled1.ipynb
best_model_todatexceptiionfreeze40.hdf5  xception15classacc59
fakedelete/                              Xception15class.ipynb
modelos/                                 Xceptionclassnewprueba.ipynb


In [34]:
import numpy as np
ypred=np.argmax(prob,axis=1)

In [31]:
y_true=validation_generator.classes

In [36]:
tf.math.confusion_matrix(y_true,ypred)

<tf.Tensor: shape=(15, 15), dtype=int32, numpy=
array([[ 9, 13,  8, 15, 14, 35, 18,  0, 12, 25,  5, 14, 14,  9,  9],
       [15, 11,  9, 11, 14, 39, 24,  0, 11, 16,  7, 14, 14,  9,  9],
       [14, 15,  4, 14, 13, 43, 15,  0, 14, 14,  8, 19, 15,  8,  6],
       [18,  9,  2, 11, 12, 49, 19,  0, 10, 25,  5, 17, 12,  7,  4],
       [12, 10,  6,  9, 16, 58, 18,  0,  6, 26,  7, 14, 12,  8,  2],
       [12,  6, 11, 11, 11, 49, 16,  0,  9, 20,  4, 27, 15,  6,  3],
       [18, 11,  5, 15, 12, 43, 20,  0,  9, 22,  1, 16, 13,  8,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 8, 11,  8,  6,  9, 45, 18,  0, 16, 17,  9, 21, 16,  5, 11],
       [ 9, 10, 13,  6, 11, 45, 16,  0, 12, 24, 10, 16, 13,  7,  8],
       [11,  6,  8, 12, 16, 54, 18,  0,  9, 15,  7, 22, 12,  2,  8],
       [14, 11,  6, 21, 17, 36, 21,  0, 14,  8, 11, 16, 15,  3,  7],
       [ 9,  6,  5, 12, 11, 39, 12,  0, 13, 36,  9, 19, 13,  8,  8],
       [11, 15, 13, 14,  8, 37, 14,  0, 15, 20,  7, 22,

Ahora con 224 image size

In [26]:
TEST_SPLIT = 0.25
epochs = 80
#batch_size = 16
batch_size=32
MICRO_DATA = True # very small subset (just 3 groups)
SAMPLE_TRAINING = False # make train set smaller for faster iteration
IMG_SIZE = (224, 224)
LEARNING_RATE = 0.001


train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

validation_generator

train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

from keras import applications
model = applications.Xception(weights='imagenet', 
                                include_top=False, input_shape = (224,224,3))



len(model.layers)

for layer in model.layers[:80]:
    layer.trainable = False
regparameter=0.0001

x = model.output
x = Flatten()(x)
x = Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.l1_l2(l1=regparameter,l2=regparameter))(x)
x = Dropout(0.4)(x)
x = Dense(4096, activation="relu")(x)
predictions = Dense(5, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])
model_final.summary()

chkpt = ModelCheckpoint(filepath='best_model_todatexceptiionfreeze40.hdf5', save_best_only=True, save_weights_only=True)

tf.keras.callbacks.ModelCheckpoint


model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 20,callbacks=[chkpt])


Found 3984 images belonging to 5 classes.
Found 995 images belonging to 5 classes.
Found 3984 images belonging to 5 classes.
Found 995 images belonging to 5 classes.


/home/davidrendon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:69: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

Epoch 1/20
124/124 [==============================] - 438s 4s/step - loss: 3.5246 - accuracy: 0.4436 - val_loss: 1.5109 - val_accuracy: 0.6230
Epoch 2/20
124/124 [==============================] - 430s 3s/step - loss: 0.7972 - accuracy: 0.8041 - val_loss: 0.4438 - val_accuracy: 0.9242
Epoch 3/20
124/124 [==============================] - 428s 3s/step - loss: 0.5598 - accuracy: 0.9479 - val_loss: 0.7867 - val_accuracy: 0.9024
Epoch 4/20
124/124 [==============================] - 430s 3s/step - loss: 0.5551 - accuracy: 0.9590 - val_loss: 0.7500 - val_accuracy: 0.9190
Epoch 5/20
124/124 [==============================] - 429s 3s/step - loss: 0.7470 - accuracy: 0.9502 - val_loss: 0.5611 - val_accuracy: 0.9502
Epoch 6/20
124/124 [==============================] - 431s 3s/step - loss: 0.5258 - accuracy: 0.9671 - val_loss: 0.6843 - val_accuracy: 0.8609
Epoch 7/20
124/124 [==============================] - 429s 3s/step - loss: 0.4328 - accuracy: 0.9808 - val_loss: 0.6522 - val_accuracy: 0.9398